In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import numpy as np
from tensorflow.keras.preprocessing import image
import os

In [3]:
# Constants
IMAGE_SIZE = (150, 150)  # Image dimensions
BATCH_SIZE = 32
EPOCHS = 20

In [4]:
# Paths to dataset folders
train_dir = 'soil_dataset_unzipped/soil_dataset/train'  # Replace with the path to your train folder
test_dir = 'soil_dataset_unzipped/soil_dataset/test'    # Replace with the path to your test folder
validation_dir = 'soil_dataset_unzipped/soil_dataset/valid'  # Replace with the path to your validation folder

In [5]:
# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,  # Normalize pixel values to [0, 1]
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1.0/255.0)  # Only rescale for test and validation


In [6]:
# Load training data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Load validation data
validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)


Found 1182 images belonging to 7 classes.
Found 141 images belonging to 7 classes.


In [7]:
# Load test data
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

Found 153 images belonging to 7 classes.


In [8]:
# Get class names
class_names = list(train_generator.class_indices.keys())
print("Class Names:", class_names)

# Save class names for future use
np.save('class_names.npy', class_names)


Class Names: ['Alluvial Soil', 'Black Soil', 'Cinder Soil', 'Laterite Soil', 'Peat Soil', 'Red Soil', 'Yellow Soil']


In [9]:
# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),  # Dropout to prevent overfitting
    Dense(len(class_names), activation='softmax')  # Output layer
])

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

c:\Users\MSI\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    epochs=EPOCHS
)

c:\Users\MSI\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 55s 1s/step - accuracy: 0.3120 - loss: 1.6744 - val_accuracy: 0.4531 - val_loss: 1.5386
Epoch 2/20
 1/36 ━━━━━━━━━━━━━━━━━━━━ 5s 169ms/step - accuracy: 0.5625 - loss: 0.9173

c:\Users\MSI\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.5625 - loss: 0.9173 - val_accuracy: 0.4531 - val_loss: 1.5308
Epoch 3/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 15s 408ms/step - accuracy: 0.6005 - loss: 0.9459 - val_accuracy: 0.5703 - val_loss: 1.3106
Epoch 4/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.6875 - loss: 0.7756 - val_accuracy: 0.5156 - val_loss: 1.3777
Epoch 5/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 12s 330ms/step - accuracy: 0.7074 - loss: 0.7932 - val_accuracy: 0.5703 - val_loss: 1.4899
Epoch 6/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.7188 - loss: 0.6036 - val_accuracy: 0.5703 - val_loss: 1.5413
Epoch 7/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 13s 365ms/step - accuracy: 0.7321 - loss: 0.6963 - val_accuracy: 0.5703 - val_loss: 1.4588
Epoch 8/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.7812 - loss: 0.5704 - val_accuracy: 0.5703 - val_loss: 1.4534
Epoch 9/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 14s 398ms/step - accuracy: 0.6979 - loss: 0.6913 - val_accuracy: 0.5156 - val_

In [11]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 795ms/step - accuracy: 0.6866 - loss: 0.9975
Test Accuracy: 64.71%


In [12]:
# Save the model
model.save('soil_classification_model.h5')
print("Model saved as soil_classification_model.h5")

Model saved as soil_classification_model.h5


In [13]:
# Function to predict soil type
def predict_soil_type(img_path, model, class_names, target_size=(150, 150)):
    img = image.load_img(img_path, target_size=target_size)  # Load and resize the image
    img_array = image.img_to_array(img)  # Convert to numpy array
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = img_array / 255.0  # Normalize pixel values to [0, 1]

    # Predict probabilities
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)[0]  # Get the predicted class index
    confidence = np.max(predictions)  # Get the confidence score

    # Map class index to class name
    predicted_class_name = class_names[predicted_class]

    return predicted_class_name, confidence

# Example usage of the predict function
new_image_path = 'sand.jpg'  # Replace with the path to your test image
predicted_class, confidence = predict_soil_type(new_image_path, model, class_names)
print(f"Predicted Class: {predicted_class}")
print(f"Confidence: {confidence * 100:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
Predicted Class: Cinder Soil
Confidence: 91.25%
